In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import transform, unary_union

def max_distance_within_polygon(polygon, point):
    """
    Find the maximum distance from a point to any location within a polygon.
    
    Args:
        polygon: GeoSeries or Polygon geometry
        point: Point geometry within the polygon
        
    Returns:
        tuple: (maximum distance, Point at maximum distance)
    """
    
    # Get polygon boundary points
    boundary = polygon.boundary
    
    # Create a dense set of points along the boundary
    # Get equally spaced points along the boundary
    distances = np.linspace(0, boundary.length, num=1000)
    boundary_points = [boundary.interpolate(distance) for distance in distances]
    
    # Calculate distances to all boundary points
    distances = [p.distance(point) for p in boundary_points]
    max_dist = max(distances)
    max_point = boundary_points[distances.index(max_dist)]
    
    return max_dist, max_point

def calculate_distances(row):
    max_dist, farthest_point = max_distance_within_polygon(row['geometry_voting_district'], row['geometry_voting_place'])
    return pd.Series({'max_dist': max_dist})

def plot_geometries(point = None, polygon = None,buffer=None):
    """
    Plot a Shapely Point and Polygon on the same matplotlib figure
    
    Parameters:
    point: shapely.geometry.Point
    polygon: shapely.geometry.Polygon
    """
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot the polygon
    if polygon is not None:
        x, y = polygon.exterior.xy
        ax.plot(x, y, 'b-', linewidth=2, label='Polygon')
        ax.fill(x, y, alpha=0.3, fc='b')
    
    # Plot the point
    if point is not None:
        ax.plot(point.x, point.y, 'ro', markersize=10, label='Point')
    
    # Plot the buffer
    if buffer is not None:
        x, y = buffer.exterior.xy
        ax.plot(x, y, 'b-', linewidth=2, label='Polygon')
        ax.fill(x, y, alpha=0.3, fc='y')
    
    # Add labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Voting District and Polling Place')
    
    
    # Equal aspect ratio
    ax.set_aspect('equal')
    
    # Add grid
    ax.grid(True)
    
    return fig, ax


### Import the Files

In [ ]:
voting_bounds = gpd.read_file('voting_bounds.geojson')
voting_places = gpd.read_file('polling_places.geojson')

voting_bounds = voting_bounds.to_crs('EPSG:2272')
voting_places = voting_places.to_crs('EPSG:2272')

census_data = gpd.read_file('tl_2024_42_bg.zip')
census_population = pd.read_csv('ACSDT5Y2022.B01003-Data.csv')
census_data = pd.merge(census_data,census_population,left_on='GEOIDFQ',right_on='GEO_ID')
census_data = census_data[census_data['COUNTYFP']=='003']
census_data = census_data.to_crs('EPSG:2272')

### Plot the data

### Step: Merge the two datasets

In [ ]:
# YOUR CODE HERE


# Include this in arguments: suffixes=('_voting_district', '_voting_place')

merged = 

### Plot district and polling place

### Step: Calculate the distances and sort

In [ ]:
# student step
merged['max_dist'] = 
merged = 

### Plot the farthest

### Plot whole county

### Create some threshold for walking distance

In [ ]:
#lets just get the portion of the geometries that are outside walking distance
walking_time = 15 #minutes
walking_speed = 3 #mph
walking_distance = walking_speed*walking_time/60*5280 #feet (unit of crs)

### Step: Create a buffer from the points

In [ ]:
#student step
merged['walking_radius'] = 

### Visualize results

### Step: Subtract the walking radius from the shapes

In [ ]:
## YOUR CODE HERE
merged['geometry_outside_walking'] = 

### View results

### Census shapefiles and populations

In [ ]:
census_data.iloc[0]

### Calculate area in sq mi and pop density

In [ ]:
census_data['area_sqmi'] = census_data['ALAND']*(1/2589988.110336)
census_data['pop'] = census_data['B01003_001E'].astype(int)
census_data['mean_density'] = census_data['pop']/census_data['area_sqmi']

### Overlay the non-walking areas with the block groups (which have population)

In [ ]:
outside_walking = gpd.GeoDataFrame(geometry=merged['geometry_outside_walking'])

# YOUR CODE HERE
census_beyond_walking = 

### Estimate the population of the remaining areas

In [ ]:
# print('County population: ',census_data['pop'].sum())
# print('Population beyond walk distance: ',int(census_beyond_walking['outside_pop_est'].sum()))
# print('Pct beyond walk distance:', int(census_beyond_walking['outside_pop_est'].sum())/census_data['pop'].sum()*100)